# Obtención de los datos

Usaremos el dataset de trashnet que es un conjunto de datos que tiene alrededor de 2500 imágenes de seis tipos diferentes de basura: vidrio, papel, cartón, plástico, metal y basura.

```consola
git clone https://github.com/garythung/trashnet
```

## Dependencias

In [ ]:
import zipfile
import os
import numpy as np

## Proceso

In [ ]:
# Descomprimir el zip con la data

data_path: str = './trashnet/data'
zip_path: str = './trashnet/data/dataset-resized.zip'
zip_ref: zipfile.ZipFile = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall(data_path)
zip_ref.close()

In [ ]:
# Separamos entre entrenamiento y validación
validation_path: str = './data/valid'
train_path: str = './data/train'

if not os.path.exists(validation_path):
    os.makedirs(validation_path)
if not os.path.exists(train_path):
    os.makedirs(train_path)

unzipped_data_path = './trashnet/data/dataset-resized'

for file in os.listdir(unzipped_data_path):
    # Se crean los nuevos directorios para data de entrenamiento y validación
    if file != '.DS_Store':
        if not os.path.exists(os.path.join(train_path, file)):
            os.makedirs(os.path.join(train_path, file))
        if not os.path.exists(os.path.join(validation_path, file)):
            os.makedirs(os.path.join(validation_path, file))

for file in os.listdir(unzipped_data_path):
    if file != '.DS_Store':
        # Extrae el 20% de las imágenes para validación (siguiendo la "regla" de 80/20)
        for subfile in os.listdir(os.path.join(unzipped_data_path, file)):
            if subfile != '.DS_Store':
                if np.random.rand(1) < 0.2:
                    os.rename(os.path.join(unzipped_data_path, file, subfile), os.path.join(validation_path, file, subfile))
                else:
                    os.rename(os.path.join(unzipped_data_path, file, subfile), os.path.join(train_path, file, subfile))

Verificamos que cada clase tenga la misma cantidad de imágenes

In [ ]:
root_directory = "./data/"

image_extension = ".jpg"  # Change the extension if necessary

train_subdirectories = [os.path.join(root_directory, "train", subdirectory) for subdirectory in os.listdir(os.path.join(root_directory, "train"))]
valid_subdirectories = [os.path.join(root_directory, "valid", subdirectory) for subdirectory in os.listdir(os.path.join(root_directory, "valid"))]

train_min_image_count = min([len([file for file in os.listdir(subdirectory) if file.endswith(image_extension)]) for subdirectory in train_subdirectories])
valid_min_image_count = min([len([file for file in os.listdir(subdirectory) if file.endswith(image_extension)]) for subdirectory in valid_subdirectories])

for subdirectory in train_subdirectories:
    image_files = [file for file in os.listdir(subdirectory) if file.endswith(image_extension)]
    image_count = len(image_files)
    difference = image_count - train_min_image_count

    for i in range(difference):
        image_to_remove = image_files[i]
        image_to_remove_path = os.path.join(subdirectory, image_to_remove)
        os.remove(image_to_remove_path)

    remaining_image_count = len([file for file in os.listdir(subdirectory) if file.endswith(image_extension)])
    print(f"Remaining image count in {subdirectory}: {remaining_image_count}")

for subdirectory in valid_subdirectories:
    image_files = [file for file in os.listdir(subdirectory) if file.endswith(image_extension)]
    image_count = len(image_files)
    difference = image_count - valid_min_image_count

    for i in range(difference):
        image_to_remove = image_files[i]
        image_to_remove_path = os.path.join(subdirectory, image_to_remove)
        os.remove(image_to_remove_path)

    remaining_image_count = len([file for file in os.listdir(subdirectory) if file.endswith(image_extension)])
    print(f"Remaining image count in {subdirectory}: {remaining_image_count}")



# Desarrollo

### Dependencias

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from keras.models import load_model
from keras import regularizers

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Preprocesamiento de los datos

In [ ]:
# Se definen los sets de entrenamiento y validación
train_data_generator: ImageDataGenerator = ImageDataGenerator(
    rescale=1.0/255.0,
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='nearest',
    shear_range=0.3,
    zoom_range=0.4
)
    
val_data_generator: ImageDataGenerator = ImageDataGenerator(rescale=1./255)

# Crea los generadores de data para entrenamiento y validación 
train_generator = train_data_generator.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=160,
    class_mode='categorical',
    shuffle=True  # Se mezcla la data de entrenamiento
)

val_generator = val_data_generator.flow_from_directory(
    validation_path,
    target_size=(150, 150),
    batch_size=160,
    class_mode='categorical',
    shuffle=False # No se mezcla la data de validación 
)

Mostramos algunas de las imagenes post-procesamiento

In [ ]:
# Obtén un lote de imágenes y etiquetas del generador de entrenamiento
images, labels = next(train_generator)

# Define los nombres de las clases en el mismo orden que se utilizó al crear el generador
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Muestra las primeras 20 imágenes
for i in range(20):
    plt.subplot(4, 5, i+1)
    plt.imshow(images[i])
    plt.title(class_names[labels[i].argmax()])
    plt.axis('off')

plt.tight_layout()
plt.show()

## Creación del modelo	

Utilizamos el modelo base de ResNet101V2  y le hacemos un fine-tuning para que se adapte a nuestro problema.

In [ ]:
# Create a base model using a pre-trained ResNet50V2 model
# Create a base model using a pre-trained ResNet101V2 model
base_model = tf.keras.applications.ResNet101V2(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
# add another layer
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(6, activation='softmax')(x)

# Define the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

### Pre-ajuste

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Pass the callback to the fit method
pre_fine_tuning_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stop]
)

### Fine-tuning

In [ ]:
# Unfreeze the last few layers of the base model
for layer in base_model.layers[-20:]:  # Increase the number of layers to fine-tune
    if not isinstance(layer, layers.BatchNormalization):  # Don't apply regularization to BatchNormalization layers
        layer.kernel_regularizer = regularizers.l2(0.01)  # Add L2 regularization
    layer.trainable = True

# Recompile the model (necessary after changing layer trainability)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # Experiment with the learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Define the early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Continue training the model with fine-tuning
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stop]
)


## Post-entrenamiento

In [ ]:
# Guardamos el resultado del modelo
if not os.path.exists('model'):
    os.makedirs('model')
model.save('./model/model/ResNet101V2_TrashClassifierV1.h5')

In [2]:
# Carga el modelo desde el archivo
model = load_model('./model/ResNet101V2_TrashClassifierV1.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 77, 77, 64)   0           ['conv1_conv[0][0]']         

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'r', label='Precision de entrenamiento')
plt.plot(epochs, val_accuracy, 'b', label='Precision de validacion')
plt.title('Valor de la precision de entrenamiento y validacion')
plt.legend(loc=0)
plt.figure()
plt.show()


In [ ]:
# plot loss
plt.plot(epochs, loss, 'r', label='Perdida de entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Perdida de validacion')
plt.title('Perdida de entrenamiento y validacion')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# get the labels of the test images.
test_labels = val_generator.classes

# make a prediction
predictions = model.predict(val_generator)

# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

errors = np.where(predicted_classes != test_labels)[0]
print("Número de errores = {}/{}".format(len(errors),val_generator.samples))
accuracy = (val_generator.samples-len(errors))/val_generator.samples
print("Precisión = ", accuracy*100, "%")

cm = confusion_matrix(test_labels, predicted_classes)
cm_plot_labels = ['cartón','vidrio','metal','papel','plastico','basura (miscelaneo)']

plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=cm_plot_labels, yticklabels=cm_plot_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
load_img = tf.keras.preprocessing.image.load_img

# show some misclassified examples
for iterator in range(len(errors)):
    ax, fig = plt.subplots(1,1)
    pred_class = np.argmax(predictions[errors[iterator]])
    pred_label = cm_plot_labels[pred_class]

    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        cm_plot_labels[test_labels[errors[iterator]]], pred_label, predictions[errors[iterator]][pred_class])

    original = load_img('{}/{}'.format(validation_path, val_generator.filenames[errors[iterator]]))
    plt.imshow(original)
    plt.title(title)
    plt.show()
    if iterator == 9:
        break

## Pruebas

In [4]:
from src.predict_image import predict_image

# Example usage:
img_path = './data/prueba_2.jpg'  
predicted_class = predict_image(model, img_path)
predicted_class

1/1 [==============================] - 0s 66ms/step


'plastic'